In [10]:
import geopandas as gpd
import pandas as pd
from pathlib import Path
# data_path = Path('data')
config = pd.read_excel('data/config.xlsx', sheet_name='layers')
config.head(3)

,Name,shape_type,color,size,alpha,label,file_type,shp,gdb,layer,hide
0,Proposed Pipeline,line,orange,4.0,1.00,label,gdf,NaN,NaN,NaN,NaN
1,APNs,filled_polygon,white,NaN,0.05,label,gdf,NaN,NaN,NaN,NaN
2,District Boundary,hollow_polygon,black,15.0,1.00,WDNAME,shp,\\ppeng.com\pzdata\clients\Arvin-Edison WSD-12...,NaN,NaN,NaN


In [11]:
def get_layer(row):
	if row['file_type'] == "shp":
		gdf = gpd.read_file(Path(row["shp"]))
		
	elif row['file_type'] == "gdb":
		gdb = Path(row["gdb"])
		gdf = gpd.read_file(gdb,layer=row["layer"])
		if row['Name'] == "Proposed Turnout":
			gdf[row['label']] = gdf[row['label']].fillna("Proposed Turnout")
	else:
		# gdf = row["gdf"]
		return None
	# gpd.GeoDataFrame(
	# 		# row["gdf"],geometry=row["gdf"].geometry
	# 		)
	
	gdf["color"] = row["color"]
	# gdf["label"] = gdf[row["label"]].astype(str)
	gdf["label"] = gdf[row["label"]].astype(str)
	gdf['shape_type'] = row['shape_type']
	gdf['alpha'] = row['alpha']
	gdf['size'] = row['size']


	gdf["layer"] = row['Name']
	print(row['Name'])
	# print(gdf.crs)
	
	
	return gdf[['size','color','shape_type','alpha','label',"layer","geometry"]]#.to_crs(epsg=4326)

# Create gdb
gdfs = {y['Name']:get_layer(y) for i,y in config.iterrows()}
# gdfs = {y['Name']:y for i,y in config.iterrows()}
# gdfs['Proposed Pipeline']

District Boundary
AEWSD North Canal
FFPPP Discharge Pipeline
Frick Unit Service Area
Proposed Turnout
DiGiorgio Pipeline
DiGiorgio Service Area
Existing Laterals
Frick Unit Pipeline
Panama Unit Service Area
Groundwater Service Area
Landowner Wells
Sandrini Unit proposed pipeline
Sandrini Unit Service Area


In [12]:
epsg = 4326
# epsg = 26745
# epsg = 2229
# epsg = 6424
gdf = pd.concat([gdf.to_crs(epsg=epsg) for gdf in gdfs.values() if gdf is not None])
gdf.to_parquet("data\\gdf-2023-08-15.parquet")